# Processing images into chips

Planet imagery was originally processed into larger tiles of 2368 x 2358 pixels at a resolution of 0.000025$^\circ$. Labelling was undertaken on only a subset of each tile, corresponding to a 0.005$^\circ$ target (~550 m). For release, the imagery was cropped to the target box and resampled to make chips of 224x224 pixels, with labels rasterized to the same dimensions. Functions provided by the built in `MakeLabels` class are used for the task. 

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import rioxarray as rxr
from datetime import datetime as dt
from makelabels import MakeLabels

## Setup

### Paths

In [21]:
root_dir = os.environ["HOME"]
proj_dir = Path(root_dir) / "projects/lacunalabels"
data_dir = Path(root_dir) / "data"
image_dir = Path(os.path.dirname(root_dir)) / \
    "data/imagery/planet/tiles" # input directory
chip_dir = Path(data_dir) / "labels/lacuna/images"  # output
log_file = str(Path(root_dir) / "logs/image-chipping-1")

for d in [chip_dir, label_dir]:
    if not os.path.isdir(d):
        os.makedirs(d)

### Catalogs

Read in assignment catalog containing the image tile names, drop duplicated site names (there are more than one assignment mapped for many sites)

In [22]:
catalog = pd.read_csv(Path(proj_dir) /\
                      "data/interim/assignments_full_wtiles.csv")
chip_catalog = (
    catalog[["name", "image_date", "x", "y", "destfile"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

## Run chipping

Using the `image_chipper` provided by `MakeLabels` run in parallel mode. 

### Initialize the class

Which also initiates a logger

In [18]:
mkl = MakeLabels(logfile=log_file)

Started dataset creation


### Define arguments for chipping function

Using a dictionary of keyword arguments, to enable parallelized implementation of the `image_chipper` function. Arguments including the half-width of the chipping target, in decimal degrees, the desired output dimensions (224x224), the input and output directories, etc. 

See `help(mkl.image_chipper)` for more details on arguments.`

In [24]:
kwargs = {
    "src_dir": image_dir, 
    "dst_dir": chip_dir, 
    "src_col": "destfile",
    "date_col": "image_date",
    "w": 0.0025, 
    "rows": 224,
    "cols": 224, 
    "crs": "epsg:4326",
    "verbose": False,
    "overwrite": False
}

### Run chipping in parallel


In [19]:
%%time
catalogf = mkl.run_parallel_thread(
    chip_catalog, mkl.image_chipper, kwargs, 4
)
catalogf = pd.DataFrame(catalogf)

Completed run
CPU times: user 2.45 s, sys: 740 ms, total: 3.19 s
Wall time: 4.8 s


In [30]:
catalogf.head()

,name,image_date,x,y,destfile,image
0,ET0007182,2017-08-15,39.5115,14.4825,2017/tile430022_2017-06_2017-11_buf184_cog.tif,ET0007182_2017-08.tif
1,NE3372442,2021-08-15,9.1515,14.4025,2021/tile417654_2021-08_buf184_cog.tif,NE3372442_2021-08.tif
2,SN0105655,2020-02-15,-16.3435,15.6325,2020/tile360764_2019-12_2020-05_buf184_cog.tif,SN0105655_2020-02.tif
3,SD4068077,2022-03-15,33.3765,14.3925,2022/tile427259_2022-03_buf184_cog.tif,SD4068077_2022-03.tif
4,ML2303293,2021-04-15,-2.3085,16.7775,2021/tile343505_2021-04_buf184_cog.tif,ML2303293_2021-04.tif


In [32]:
catalogf.reset_index(drop=True, inplace=True)
catalogf.drop(columns=["destfile", "x", "y"], inplace=True)
catalogf.head()

,name,image_date,image
0,ET0007182,2017-08-15,ET0007182_2017-08.tif
1,NE3372442,2021-08-15,NE3372442_2021-08.tif
2,SN0105655,2020-02-15,SN0105655_2020-02.tif
3,SD4068077,2022-03-15,SD4068077_2022-03.tif
4,ML2303293,2021-04-15,ML2303293_2021-04.tif


In [41]:
catalog_out = pd.merge(catalog, catalogf, how="left")
catalog_out.to_csv(
    Path(proj_dir) / "data/interim/label_catalog_int.csv", index=False
)
catalog_out.head()

,name,assignment_id,Type,Labeller,label_time,completion_time,farea,nflds,status,Score,...,Qscore,QN,Rscore,x,y,tile,Class,image_date,destfile,image
0,ET0007182,50.0,F,22,1.416667,2023-11-13T19:09:18Z,NaN,NaN,Untrusted,NaN,...,0.621788,0.361483,0.706386,39.5115,14.4825,430022,2,2017-08-15,2017/tile430022_2017-06_2017-11_buf184_cog.tif,ET0007182_2017-08.tif
1,NE3372442,51.0,F,22,0.900000,2023-11-13T19:10:12Z,NaN,NaN,Untrusted,NaN,...,0.621788,0.361483,0.706386,9.1515,14.4025,417654,2,2021-08-15,2021/tile417654_2021-08_buf184_cog.tif,NE3372442_2021-08.tif
2,SN0105655,48.0,F,13,10.566667,2023-11-13T19:12:41Z,3.216725,8.0,Untrusted,NaN,...,0.623301,0.348254,0.728205,-16.3435,15.6325,360764,2,2020-02-15,2020/tile360764_2019-12_2020-05_buf184_cog.tif,SN0105655_2020-02.tif
3,SD4068077,52.0,F,22,2.616667,2023-11-13T19:12:49Z,6.902422,7.0,Untrusted,NaN,...,0.621788,0.361483,0.706386,33.3765,14.3925,427259,2,2022-03-15,2022/tile427259_2022-03_buf184_cog.tif,SD4068077_2022-03.tif
4,ML2303293,16.0,F,22,126.450000,2023-11-09T17:06:38Z,NaN,NaN,Untrusted,NaN,...,0.621788,0.361483,0.706386,-2.3085,16.7775,343505,2,2021-04-15,2021/tile343505_2021-04_buf184_cog.tif,ML2303293_2021-04.tif
